In [1]:
!pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, genomics_ood, german_credit

In [3]:
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train",with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9JVC6L/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9JVC6L/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
imdb_test = tfds.load(name="imdb_reviews", split="test",as_supervised=True)

In [5]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [6]:
for example, label in imdb_train.take(1):
  print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [7]:
## Normalization and vectorization

In [9]:
tokenizer = tfds.deprecated.text.Tokenizer()

In [10]:
tokenizer.tokenize('hey how are you?')

['hey', 'how', 'are', 'you']

In [11]:
vocabulary_set = set()
MAX_TOKENS = 0

In [12]:
for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
    MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [13]:
MAX_TOKENS

2525

In [15]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set,tokenizer=tokenizer)

In [16]:
vocab_size = imdb_encoder.vocab_size

In [18]:
print(vocab_size, MAX_TOKENS)

93931 2525


#### The vocabulary has 93,931 tokens. The longest review has 2,525 tokens.

LSTMs expect sequences of equal length. Padding and truncating operations make reviews of equal length.

In [25]:
for example, label in imdb_train.take(1):
  print(example)
  encoded = imdb_encoder.encode(example.numpy())
  print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [ ]:
## Note that punctuation is removed from these reviews when they are reconstructed from the encoded representations.

In [26]:
## imdb_encoder.save_to_file("reviews_vocab")

In [27]:
## To load the encoder from the file and test it

In [ ]:
## enc = tfds.features.text.TokenTextEncoder.load_from_file("reviews_vocab")
## enc.decode(enc.encode("Good case. Excellent value."))

Tokenization: Reviews need to be tokenized into words.

Encoding: These words need to be mapped to integers using the vocabulary.

Padding: Reviews can have variable lengths, but LSTMs expect vectors of
the same length. So, a constant length is chosen. Reviews shorter than this
length are padded with a specific vocabulary index, usually 0 in TensorFlow.

In [29]:
from tensorflow.keras.preprocessing import sequence

In [30]:
def encode_pad_transform(text):
  encodded = imdb_encoder.encode(text.numpy())
  pad = sequence.pad_sequences([encoded],padding="post",
                               maxlen = 150)
  return np.array(pad[0],dtype = np.int64)

In [31]:
def encode_tf_fn(text,label):
  encoded = tf.py_function(encode_pad_transform,
                           inp=[text],Tout=(tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded , label

In [32]:
subset = imdb_train.take(10)

In [34]:
for text,label in subset:
  print(text,label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on t

In [35]:
test = subset.map(encode_tf_fn)

In [36]:
test

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [38]:
for text,label in test.take(1):
  print(text,label)
  print(imdb_encoder.decode(text))

tf.Tensor(
[49680 43137 72923 20792 11678 93143 33663 35617 64669 41298 76938 41446
  3876 79316 80764 79027 28707 55222 16008 49310 12873 92170 87729 80382
 14970 64669 26073 88282 82378 76938 81388 63812 26073 49310 12556  6567
 41465 12671 87729 93143 42841 77590  6620 49680 93143 82099 72923 48337
 28261 70178 64859 11004 55089 19736 88849 40604 76794 63360 26698  2967
 62179 11683 76794 57236 26073 21636 18555 72512 30660 37916 38153 18774
   946 87247 22323 86092 25437 61389 21986 79316 43137 19536 92170 30247
 88849 86997  2592 76938 30247 93143 41873 43137 47432 12251 75286 53735
 16628 62740 80685 48127 41873 55062 16008 71084 63132 87729 72876 33212
 42841 63132  3876 79316 42841 14933 54736 62740  6567 50982 35769 49016
 74593     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [60]:
encoded_train = imdb_train.map(encode_tf_fn)
encoded_test = imdb_test.map(encode_tf_fn)

In [98]:
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [61]:
## LSTM model with embeddings

In [99]:
def lstm(vocab_size,embedding_dim,rnn_units,batch_size):
  model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,
                                  embedding_dim,
                                  mask_zero = True,
                                  batch_input_shape = [batch_size,None]
                                  ),
        tf.keras.layers.LSTM(rnn_units),
        tf.keras.layers.Dense(1,activation = "sigmoid")
  ])

  return model

In [100]:
vocab_size = imdb_encoder.vocab_size

In [101]:
print(vocab_size)

93931


In [102]:
# The embedding dimension
embedding_dim = 64

In [103]:
# Number of RNN units
rnn_units = 64

In [104]:
# batch size
BATCH_SIZE=100

In [105]:
model = lstm(vocab_size = vocab_size,
             embedding_dim = embedding_dim,
             rnn_units = rnn_units,
             batch_size=BATCH_SIZE)

In [106]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (100, None, 64)           6011584   
                                                                 
 lstm_10 (LSTM)              (100, 64)                 33024     
                                                                 
 dense_10 (Dense)            (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy', 'Precision', 'Recall'])

In [110]:
## the dataset needs to be batched and training can be started
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [111]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
129/250 [==============>...............] - ETA: 40s - loss: 0.6935 - accuracy: 0.4903 - precision: 0.4906 - recall: 0.4942

KeyboardInterrupt: ignored

In [74]:
def bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            mask_zero=True,
                            batch_input_shape=[batch_size, None]),
  tf.keras.layers.Dropout(0.8),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [93]:
model = bilstm(vocab_size = vocab_size,
             embedding_dim = 64,
             rnn_units = 32,
             batch_size=50)

In [94]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (50, None, 64)            6011584   
                                                                 
 dropout_12 (Dropout)        (50, None, 64)            0         
                                                                 
 bidirectional_6 (Bidirectio  (50, 64)                 24832     
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (50, 64)                  0         
                                                                 
 dense_9 (Dense)             (50, 1)                   65        
                                                                 
Total params: 6,036,481
Trainable params: 6,036,481
Non-trainable params: 0
____________________________________________

In [95]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])

In [96]:
## the dataset needs to be batched and training can be started
encoded_train_batched = encoded_train.batch(50)

In [97]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
500/500 [==============================] - 137s 260ms/step - loss: 0.6938 - accuracy: 0.4918 - precision: 0.4915 - recall: 0.4728
Epoch 2/10
500/500 [==============================] - 130s 259ms/step - loss: 0.6936 - accuracy: 0.4927 - precision: 0.4927 - recall: 0.4922
Epoch 3/10
281/500 [===============>..............] - ETA: 56s - loss: 0.6934 - accuracy: 0.4993 - precision: 0.4989 - recall: 0.5581

KeyboardInterrupt: ignored

In [112]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [113]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [114]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (50, None, 128)           12023168  
                                                                 
 dropout_14 (Dropout)        (50, None, 128)           0         
                                                                 
 bidirectional_7 (Bidirectio  (50, None, 128)          98816     
 nal)                                                            
                                                                 
 dropout_15 (Dropout)        (50, None, 128)           0         
                                                                 
 bidirectional_8 (Bidirectio  (50, 128)                98816     
 nal)                                                            
                                                                 
 dropout_16 (Dropout)        (50, 128)               

In [115]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [116]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)


In [117]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 391s 752ms/step - loss: 0.6937 - accuracy: 0.4959 - precision: 0.4952 - recall: 0.4277
Epoch 2/5
500/500 [==============================] - 384s 766ms/step - loss: 0.6935 - accuracy: 0.4986 - precision: 0.4985 - recall: 0.4663
Epoch 3/5
500/500 [==============================] - 376s 750ms/step - loss: 0.6934 - accuracy: 0.5027 - precision: 0.5030 - recall: 0.4550
Epoch 4/5
500/500 [==============================] - 375s 749ms/step - loss: 0.6935 - accuracy: 0.4960 - precision: 0.4956 - recall: 0.4465
Epoch 5/5
500/500 [==============================] - 380s 758ms/step - loss: 0.6934 - accuracy: 0.4972 - precision: 0.4970 - recall: 0.4681
